# Configuración del entorno

In [5]:
import sys
from pathlib import Path

# Añadir la carpeta raíz del proyecto al path
sys.path.append(str(Path.cwd().parent))

# Verificación opcional
print("📁 Ruta añadida al path:", Path.cwd().parent)


📁 Ruta añadida al path: /Users/leidygarciaguzman/engie_demand_forecast


# Carga de variables de entorno (API Key)

In [6]:
from dotenv import load_dotenv
import os

# 🟡 Ruta real del .env desde la carpeta del notebook
load_dotenv(dotenv_path="../visualizations/.env")

# Obtener la clave
API_KEY = os.getenv("AEMET_API_KEY")

# Verificación
if API_KEY:
    print("✅ API Key cargada correctamente:", API_KEY[:10] + "********")
else:
    print("❌ No se pudo cargar la API Key.")


✅ API Key cargada correctamente: eyJhbGciOi********


# Definición de la función para consultar AEMET

In [7]:
import requests
import pandas as pd

def get_aemet_weather(api_key: str, station_id: str, date: str) -> pd.DataFrame:
    """
    Descarga los datos meteorológicos horarios para una estación de AEMET.
    :param api_key: Clave API de AEMET
    :param station_id: Código de estación (ej: '3195' para Madrid - Retiro)
    :param date: Fecha en formato 'YYYY-MM-DD'
    :return: DataFrame con datos horarios
    """
    # Paso 1: URL de metadatos para obtener el enlace de descarga real
    url = f"https://opendata.aemet.es/opendata/api/observacion/convencional/datos/estacion/{station_id}/?api_key={api_key}"

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"❌ Error en AEMET (paso metadatos): {response.status_code}")

    json_data = response.json()
    data_url = json_data.get("datos")

    if not data_url:
        raise Exception("❌ No se encontró la URL de descarga en la respuesta de AEMET.")

    # Paso 2: Descargar los datos desde esa URL
    data_response = requests.get(data_url)
    if data_response.status_code != 200:
        raise Exception(f"❌ Error al descargar datos: {data_response.status_code}")

    raw_data = data_response.json()
    df = pd.DataFrame(raw_data)

    # Filtrar por fecha
    df = df[df['fint'].str.startswith(date)]

    return df

In [8]:
from src.extract_aemet import get_aemet_weather

df = get_aemet_weather(API_KEY, station_id="3195", date="2024-07-01")
df.head(5)


/Users/leidygarciaguzman/engie_demand_forecast/src/extract_aemet.py:34: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(data_response.text)


,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,horaracha,presMax,horaPresMax,presMin,horaPresMin,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2024-07-01,3195,"MADRID, RETIRO",MADRID,667,"24,1","0,0","18,2",05:10,"30,0",...,21:50,"943,7",23,"938,5",3,40,69,04:30,24,17:10


In [15]:
import pandas as pd
import glob

# Buscar los CSVs
csv_files = glob.glob("../data/Custom-Report-2025-07-*.csv")

# Nombres de columnas esperadas
column_names = ["datetime", "real_mw", "forecast_mw", "scheduled_mw"]

# Leer los archivos con tolerancia a errores
df_list = []
for file in csv_files:
    df = pd.read_csv(
        file,
        sep=",",
        names=column_names,
        skiprows=1,
        encoding="latin-1",
        on_bad_lines='skip'  # <- Ignora líneas con problemas
    )
    df_list.append(df)

# Unir todos los DataFrames
df_ree = pd.concat(df_list, ignore_index=True)


In [17]:
# Convertir datetime
df_ree['datetime'] = pd.to_datetime(df_ree['datetime'], errors='coerce')

# Limpiar comillas, comas y convertir a float
for col in ['real_mw', 'forecast_mw', 'scheduled_mw']:
    df_ree[col] = (
        df_ree[col]
        .astype(str)
        .str.replace('"', '', regex=False)
        .str.replace(',', '', regex=False)
        .astype(float)
    )

# Ver las primeras filas para revisar
df_ree.head()


/var/folders/rl/7bt84j4j31gg1ssp767d9xgm0000gn/T/ipykernel_12093/2562031363.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ree['datetime'] = pd.to_datetime(df_ree['datetime'], errors='coerce')


ValueError: could not convert string to float: 'Real'

In [16]:
print(df_ree.head(5))
print("Número de filas:", len(df_ree))
print("Número de columnas:", len(df_ree.columns))
print("Columnas:", df_ree.columns.tolist())
print("Tipos de datos:")
print(df_ree.dtypes)
print("Primeras filas:")

  datetime real_mw forecast_mw scheduled_mw
0     Hora    Real    Prevista   Programada
1     Hora    Real    Prevista   Programada
2     Hora    Real    Prevista   Programada
3     Hora    Real    Prevista   Programada
4     Hora    Real    Prevista   Programada
Número de filas: 7
Número de columnas: 4
Columnas: ['datetime', 'real_mw', 'forecast_mw', 'scheduled_mw']
Tipos de datos:
datetime        object
real_mw         object
forecast_mw     object
scheduled_mw    object
dtype: object
Primeras filas:
